In [3]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
# TO USE:
# Uncomment three !pip install lines
# Run this cell ONLY
# Runtime -> Restart session
# Comment three !pip install lines
# Run all
# !pip install pyLDAvis
# !pip install --user -U nltk
# !pip install "pandas==1.5.3"

import numpy as np
import json
import glob
import nltk
import pandas as pd #to work with csv files

# pd.__version__ = 2.0.0
print(pd.__version__)

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

1.5.3


In [ ]:
# Stopwords: lists of words with little context contributions
nltk.download('stopwords')

stopwords = stopwords.words("english") #get stopwords from nltk

stopwords.append("https")
stopwords.append("com")
stopwords.append("www")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [1]:
# Removing stop words to reduce white noises in our dataset
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]

In [1]:
#  Loading input file
def load_data(file):
  data = pd.read_csv(file)
  return data

In [5]:
data = load_data('../Used_Dataset/finalCumulative.csv')['text']
data.head()

<bound method NDFrame.head of 0        Concerned parents: Roblox scams, and cash grab...
1        Roblox (the corporation) doesn't exactly contr...
2        this is something i can agree with, its seriou...
3        Thats pretty much every simulator in a nutshel...
4        Basically...\n\n\nAll these "simulator" games ...
                               ...                        
22986    Someone linked a celeron geobook for 130$ that...
22987    Ah, got it. I'd keep an eye on Walmart for che...
22988                                       Here's hoping!
22989    https://www.walmart.com/ip/Gateway-15-6-Ultra-...
22990                           PERFECT! Thank you so much
Name: text, Length: 22991, dtype: object>

In [6]:
# Removing leading or trailing white spaces
data = [d.strip() for d in data]

In [7]:
# Removing new line characters
new_data = []
for d in data:
  new_data.append(d.replace('\n', '').replace('\\', ''))

print(new_data[0])


Concerned parents: Roblox scams, and cash grab games that pretty much force children to rebuy game packs they already own.Ok, so the title says it all. I am sick of seeing all these games that are clearly a cash grab, and I have been blind to it until recently checking what my son has been begging for Robux for. Take for example Ninja Legends 2. These guys just put some shine on the original game and put a number 2 at the end of it. There is nothing new about the second game!! I just checked it myself on my sons tablet.Why do the people who run Roblox let this stuff fly? Games like this only rename their original game and call it new so it forces children to re buy the same packs they already had, and the sad part is, many of these small children fall for it and beg their parents to buy them Robux to pay for something they already got..... its highway robbery. Something needs to be done about this soon. Roblox seems to be getting worse with all the Robux scams I need to constantly moni

In [ ]:
# Lemmatization: the process of removing all 'ing', 'ed' and making different forms of a word into a single form
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ','VERB','ADV']):
  nlp = spacy.load('en_core_web_sm', disable= ['parser', "ner"])
  text_out= []
  for text in texts:
    doc= nlp(text)
    new_text= []
    for token in doc:
      if token.pos_ in allowed_postags:
        new_text.append(token.lemma_)
    final = " ".join(new_text)
    text_out.append(final)
  return text_out

In [ ]:
# Applying lemmatization on the input data
lemmatize_text= lemmatization(new_data)
print(lemmatize_text[0])

scamme there 17th daughter trick user handing weapon call promise get back share friend obviously scam then user say only get item back hand entire inventory thankfully user log shortly now miss item say take very long time get now understand believe people say thing help send few report response week recording imagine people already able view chat trade log imagine recording much advice greatly appreciate


In [ ]:
# Applying stopwords removal on input data
new_data = remove_stopwords(new_data)
print(new_data)

In [ ]:
# Tokenization: the process of converting a sentence into a vector of words
def gen_words(texts):  # tokenizing sentences into a list of words
  final = []
  for text in texts:
    new = gensim.utils.simple_preprocess(text, deacc=True)
    final.append(new)
  return(final)

data_words = gen_words(lemmatize_text)

print(data_words[0])

['scamme', 'there', 'th', 'daughter', 'trick', 'user', 'handing', 'weapon', 'call', 'promise', 'get', 'back', 'share', 'friend', 'obviously', 'scam', 'then', 'user', 'say', 'only', 'get', 'item', 'back', 'hand', 'entire', 'inventory', 'thankfully', 'user', 'log', 'shortly', 'now', 'miss', 'item', 'say', 'take', 'very', 'long', 'time', 'get', 'now', 'understand', 'believe', 'people', 'say', 'thing', 'help', 'send', 'few', 'report', 'response', 'week', 'recording', 'imagine', 'people', 'already', 'able', 'view', 'chat', 'trade', 'log', 'imagine', 'recording', 'much', 'advice', 'greatly', 'appreciate']


In [ ]:
# Converting the word vectors into number vectors. Each word will be assigned to a corresponding key value for reversion

from prompt_toolkit.completion import word_completer
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
  new = id2word.doc2bow(text)
  corpus.append(new)

print(corpus[0])

word = id2word[[0][:1][0]]
print(word_completer)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 3), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 2), (20, 2), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 2), (28, 1), (29, 2), (30, 1), (31, 1), (32, 3), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 3), (49, 1), (50, 1), (51, 1), (52, 1)]
<module 'prompt_toolkit.completion.word_completer' from '/usr/local/lib/python3.10/dist-packages/prompt_toolkit/completion/word_completer.py'>


In [ ]:
# Runnning Topic Modeling
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=5,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True,
                                       alpha=0.01,
                                       eta=0.61)

In [ ]:
from pprint import pprint
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.034*"https" + 0.030*"com" + 0.021*"www" + 0.011*"reddit" + 0.010*"users" '
  '+ 0.010*"roblox" + 0.009*"subreddit" + 0.008*"games" + 0.008*"it" + '
  '0.007*"auto"'),
 (1,
  '0.034*"game" + 0.022*"play" + 0.019*"get" + 0.017*"just" + 0.012*"so" + '
  '0.012*"know" + 0.012*"try" + 0.011*"make" + 0.011*"son" + 0.010*"say"'),
 (2,
  '0.013*"sword" + 0.007*"speed" + 0.006*"flower" + 0.005*"glow" + '
  '0.005*"spin" + 0.004*"villain" + 0.004*"throne" + 0.004*"white" + '
  '0.003*"castle" + 0.003*"armor"'),
 (3,
  '0.076*"buy" + 0.046*"card" + 0.036*"robux" + 0.036*"purchase" + '
  '0.027*"money" + 0.023*"gift" + 0.022*"item" + 0.015*"pet" + 0.015*"spend" + '
  '0.015*"trade"'),
 (4,
  '0.069*"account" + 0.024*"email" + 0.017*"password" + 0.016*"daughter" + '
  '0.015*"help" + 0.015*"support" + 0.014*"log" + 0.014*"use" + 0.014*"get" + '
  '0.013*"roblox"')]


In [ ]:
# Evaluating topics generation
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')

coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5243170850928109


In [ ]:

def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = gensim.models.LdaModel(corpus=corpus,
                                       id2word=dictionary,
                                       num_topics=k,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       alpha=a,
                                       eta=b,
                                       per_word_topics=True)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
    return coherence_model_lda.get_coherence()

In [ ]:
# Fine tuning the LDA gensim model to fine the most optimal hyperparameters 
# that would generate the best topic clusters. Mostly focusing on three specific
# variables: topic_num, alpha, beta.

import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
#A list of alpha values to be tested.
#It includes a range of values from 0.01 to 1 with a step size of 0.3, as well as two special values: 'symmetric' and 'asymmetric'.
alpha = list(np.arange(0.01, 1, 0.3))
#assumes that all documents are equally likely to be a mixture of topics, resulting in a uniform distribution of topics across documents. So it maps rreviews to each topic evenly
#It makes the model assume that every document should have roughly the same proportion of topics.
alpha.append('symmetric')
#allows for more variability in document-topic proportions.
#It doesn't enforce a uniform distribution and allows documents to have varying degrees of focus on different topics.
alpha.append('asymmetric')

# Beta parameter
#A list of beta values to be tested. It follows a similar pattern to alpha, including a range and 'symmetric'.
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets:
# We have the corpus sets, which contains 2 versions of the corpus, 15% and 20%
num_of_docs = len(corpus)

#A list containing two versions of the 15% corpus, one with 20% of the documents and another with 15% of the documents.
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.20)),
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.15))]

corpus_title = ['20% Corpus', '15% Corpus']

#Stores the results of the LDA model with different hyperparameters.
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
  #progress bar
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))

    # iterate through validation corpus sets
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    #reconnect the runtime
                    check_and_reconnect_drive()
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word,
                                                  k=k, a=a, b=b)

                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)

                    pbar.update(1)
    # pd.DataFrame(model_results).to_csv('../Result/lda_tuning_results.csv', index=False)
    # pd.DataFrame(model_results).to_csv('/content/drive/My Drive/Classes/lda_tuning_results.csv', index=False)
    # pbar.close()

In [ ]:
if 1==1:
# pd.DataFrame(model_results).to_csv('./Result/lda_tuning_results.csv', index=False)
    pd.DataFrame(model_results).to_csv('/content/drive/My Drive/Classes/lda_tuning_results.csv', index=False)
    pbar.close()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Load the model results from the CSV file
model_results_df = pd.read_csv('/content/lda_tuning_results.csv')

# Filter the results for the 75% Corpus (you can change this if needed)
# filtered_results = model_results_df[model_results_df['Validation_Set'] == '20% Corpus']

# Group the results by the number of topics (Topics) and calculate the mean coherence score
coherence_by_topics = model_results_df.groupby('Topics')['Coherence'].mean()

# Plot the coherence score against the number of topics
plt.figure(figsize=(12, 6))
plt.plot(coherence_by_topics.index, coherence_by_topics.values, marker='o', linestyle='-')
plt.xlabel('Number of Topics')
plt.ylabel('Coherence Score (C_v)')
plt.title('Coherence Score vs. Number of Topics')
plt.grid(True)
plt.show()

In [ ]:
data_topic_3 = model_results_df[model_results_df['Topics'] == 3]
max_coherence = np.array(data_topic_3['Coherence'])
print(max_coherence[3])
# print(np.argmax(max_coherence))

##Visualizing the Data


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.313448  0.030409       1        1  80.098162
4      0.147650 -0.244141       2        1   9.808519
3     -0.248842 -0.130600       3        1   5.192241
2     -0.120131  0.225943       4        1   2.902635
0     -0.092126  0.118389       5        1   1.998443, topic_info=                 Term         Freq        Total Category  logprob  loglift
145           account  2510.000000  2510.000000  Default  30.0000  30.0000
215               buy  1463.000000  1463.000000  Default  29.0000  29.0000
73             roblox  2579.000000  2579.000000  Default  28.0000  28.0000
1120             card   898.000000   898.000000  Default  27.0000  27.0000
16               help  2430.000000  2430.000000  Default  26.0000  26.0000
...               ...          ...          ...      ...      ...      ...
1812              bot    33.974705    55.189340   Topic5  -5.3841   3.4276
9730           acount    12.985621    18.386321   Topic5  -6.3459   3.5650
876          downvote    15.357096    27.467630   Topic5  -6.1782   3.3314
73             roblox    71.480009  2579.049363   Topic5  -4.6403   0.3270
13549  suggestalaptop    12.452317    21.006977   Topic5  -6.3878   3.3899

[278 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
9928      1  0.067517      ab
9928      3  0.067517      ab
9928      5  0.810201      ab
97        1  0.013183  access
97        2  0.975548  access
...     ...       ...     ...
874       5  0.986103     www
194       1  0.998935    year
194       2  0.000480    year
194       3  0.000480    year
194       4  0.000480    year

[721 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 4, 3, 1])

In [ ]:
lda_model_tuned = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=4,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True,
                                       alpha=0.01,
                                       eta=0.91)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tuned, corpus, id2word, mds='mmds', R=30)
vis

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.280517  0.153262       1        1  88.320095
0      0.032990 -0.210468       2        1   5.114915
1     -0.168347 -0.041390       3        1   4.302635
3     -0.145160  0.098596       4        1   2.262354, topic_info=          Term         Freq        Total Category  logprob  loglift
73      roblox  2912.000000  2912.000000  Default  30.0000  30.0000
215        buy  1253.000000  1253.000000  Default  29.0000  29.0000
172   purchase   806.000000   806.000000  Default  28.0000  28.0000
1120      card   771.000000   771.000000  Default  27.0000  27.0000
174      robux   604.000000   604.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
533    comment    42.644715   215.179029   Topic4  -5.2809   2.1702
73      roblox   120.839517  2912.642581   Topic4  -4.2393   0.6064
165    message    55.437051   717.220880   Topic4  -5.0185   1.2286
480    profile    40.354917   558.151098   Topic4  -5.3361   1.1618
199       link    22.421426   649.986760   Topic4  -5.9238   0.4218

[209 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
145       1  0.993090  account
145       2  0.006766  account
145       3  0.000150  account
9730      1  0.228698   acount
9730      3  0.057174   acount
...     ...       ...      ...
874       3  0.007764      www
874       4  0.986040      www
194       1  0.998840     year
194       2  0.000598     year
194       3  0.000598     year

[550 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2, 4])

In [ ]:
# Get the topic distribution for each document
document_topics = lda_model.get_document_topics(corpus)
print(document_topics)
# Filter reviews that have topic 1 as their main topic
reviews_in_topic_1 = [idx for idx, topics in enumerate(document_topics) if max(topics, key=lambda x: x[1])[0] == 1]

# Get the number of reviews in topic 1
num_reviews_in_topic_1 = len(reviews_in_topic_1)

print(f'There are {num_reviews_in_topic_1} reviews in topic 1.')

There are 15089 reviews in topic 1.


In [ ]:
print(max(document_topics))

[(4, 0.99885863)]


In [ ]:
# Filter reviews that have topic 5 as their main topic
reviews_in_topic_5 = [idx for idx, topics in enumerate(document_topics) if max(topics, key=lambda x: x[1])[0] == 5]

# Get the number of reviews in topic 5
num_reviews_in_topic_5 = len(reviews_in_topic_5)

print(f'There are {num_reviews_in_topic_5} reviews in topic 5.')

There are 0 reviews in topic 5.


In [ ]:
# Get the topic distribution for each document
document_topics = lda_model.get_document_topics(corpus)

# Filter reviews that have topic 1 as their main topic
reviews_in_topic_1 = [idx for idx, topics in enumerate(document_topics) if max(topics, key=lambda x: x[1])[0] == 1]

# Print the reviews
for idx in reviews_in_topic_1:
    print(' '.join(data_words[idx]))

Streaming output truncated to the last 5000 lines.
close enough
so tallented
well draw then ever tbh
look so great wish like
really goood
actually look really good
really cool drawing
so cuuute
good leg look fall fifth floor
badcc mean creator
awesome
even draw good
old son well draw
still draw stick figure
teach way
young relate dude last name so similar asimolowo
love young use love go showcase
erase line look just sketch
help bug month now hear music whatsoever only way able fix create new window user practical even open local video help say error code dev log
look jojo
good man
re so sweet make day
great person thank nice
go excite dad say reply drawing brighten today
make smile
thank so much really understand work even love play re try teach good balance gaming own creativity match like robux see go thank again
sir good man
too
happy cake day
happy cake day
kid draw let enjoy smh
thank ask know know explain
more pimp name slickback
fan suddenly love feel need talk sub ruin good
ru